In [1]:
import mod_sql
import pandas as pd

In [3]:
# mod_sql을 통해서 Database 클래스를 생성
db = mod_sql.Database("localhost", "root", "1234", "ubion7", 3306)

In [4]:
sql = """
    select
    `Country` as '국가',
    `Units Sold` as '판매개수',
    `Unit Price` as '가격',
    `Unit Cost` as '원가',
    (`Units Sold` * `Unit Price`) as '총수익',
    (`Units Sold` * `Unit Cost`) as '총비용',
    (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
    from
    `Sales Records`
"""

db.sql_query(sql)

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20


pandas로 같은 형태 작성 (Sales Records)

In [27]:
df = pd.read_csv("../csv/Sales Records.csv")

In [28]:
df.head(2)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,Chad,Office Supplies,Online,L,1/27/2011,292494523,2/12/2011,4484,651.21,524.96,2920025.64,2353920.64,566105.0
1,Europe,Latvia,Beverages,Online,C,12/28/2015,361825549,1/23/2016,1075,47.45,31.79,51008.75,34174.25,16834.5


In [8]:
df.columns

Index(['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Priority',
       'Order Date', 'Order ID', 'Ship Date', 'Units Sold', 'Unit Price',
       'Unit Cost', 'Total Revenue', 'Total Cost', 'Total Profit'],
      dtype='object')

In [29]:
# df.drop(['Region','Item Type', 'Sales Channel', 'Order Priority','Order Date', 'Order ID', 'Ship Date'], axis = 1, inplace = True)
# df.drop(['Total Revenue', 'Total Cost', 'Total Profit'], axis = 1, inplace = True)

df = df[['Country', 'Units Sold', 'Unit Price', 'Unit Cost']]

In [30]:
df.head(3)

,Country,Units Sold,Unit Price,Unit Cost
0,Chad,4484,651.21,524.96
1,Latvia,1075,47.45,31.79
2,Pakistan,6515,154.06,90.93


In [31]:
df.columns = ['국가', '판매개수', '가격', '원가']

In [32]:
df.head(3)

,국가,판매개수,가격,원가
0,Chad,4484,651.21,524.96
1,Latvia,1075,47.45,31.79
2,Pakistan,6515,154.06,90.93


In [33]:
# (Units Sold * Units Price) as 총수익
# (Units Sold * Units Cost) as 총비용
# (Units Sold * Units Price - Units Sold * Units Cost) as 총이윤

# 총수익
df['총수익'] = df['판매개수'] * df['가격']
# 총비용
df['총비용'] = df['판매개수'] * df['원가']

In [34]:
# 총이윤
df['총이윤'] = df['총수익'] - df['총비용']

In [35]:
df.head()

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06


## sql에서 반올림

In [37]:
sql = """
    select
    `Total Cost`,
    round(`Total Cost`, 1)
    from
    `Sales Records`
"""

db.sql_query(sql)

,Total Cost,"round(`Total Cost`, 1)"
0,2353920.64,2353920.6
1,34174.25,34174.2
2,592408.95,592409.0
3,3861014.82,3861014.8
4,110978.89,110978.9
...,...,...
9995,273298.63,273298.6
9996,147991.46,147991.5
9997,229507.32,229507.3
9998,3174991.14,3174991.1


## sql 그룹화
1. Country, Item Type을 기준으로 그룹화
2. Unit Sold의 평균, 합계 생성
3. Country를 기준으로 오름차순 정렬

In [42]:
sql = """
    select
    `Country`,
    `Item Type`,
    AVG(`Units Sold`) as AVG,
    SUM(`Units Sold`) as SUM
    from
    `Sales Records`
    group by
    `Country`,
    `Item Type`
    order by
    `Country`
    ASC
"""

db.sql_query(sql)

,Country,Item Type,AVG,SUM
0,Afghanistan,Baby Food,1368.0000,1368
1,Afghanistan,Beverages,4793.3333,14380
2,Afghanistan,Cereal,6101.8889,54917
3,Afghanistan,Clothes,5977.4000,29887
4,Afghanistan,Cosmetics,3044.2500,12177
...,...,...,...,...
2193,Zimbabwe,Meat,9640.0000,9640
2194,Zimbabwe,Office Supplies,4361.7500,17447
2195,Zimbabwe,Personal Care,9239.0000,18478
2196,Zimbabwe,Snacks,5438.0000,16314


pandas로 작성 (반올림)

In [66]:
df = pd.read_csv("../csv/Sales Records.csv")

In [50]:
df1.head(1)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,Chad,Office Supplies,Online,L,1/27/2011,292494523,2/12/2011,4484,651.21,524.96,2920025.64,2353920.64,566105.0


In [67]:
df2 = df[['Country', 'Item Type', 'Units Sold']]

In [70]:
df2.groupby(['Country', 'Item Type']).agg(['mean', 'sum'])['Units Sold'].reset_index()  # 뒤에 ['Units Sold']는 해당 인덱스 벗기는 과정
# df기준으로 한다면
# df.groupby(['Country', 'Item Type'])['Units Sold'].agg(['mean', 'sum'])

,Country,Item Type,mean,sum
0,Afghanistan,Baby Food,1368.000000,1368
1,Afghanistan,Beverages,4793.333333,14380
2,Afghanistan,Cereal,6101.888889,54917
3,Afghanistan,Clothes,5977.400000,29887
4,Afghanistan,Cosmetics,3044.250000,12177
...,...,...,...,...
2193,Zimbabwe,Meat,9640.000000,9640
2194,Zimbabwe,Office Supplies,4361.750000,17447
2195,Zimbabwe,Personal Care,9239.000000,18478
2196,Zimbabwe,Snacks,5438.000000,16314


### dept table에서 부서의 이름 (dname)이 sales인 소속 사원의 정보 출력
1. dept 테이블에서 부서이름이 sales인 deptno를 출력
2. emp 테이블에서 1번 과정에서 나온 deptno를 기준으로 사원 출력

In [72]:
db = mod_sql.Database("localhost", "root", "1234", "ubion7", 3306)

In [ ]:
sql = """
    select
    `DEPTNO`
    from
    `dept`
    where
    `DNAME` = sales
"""

